# Cvičenie 4: Value Iteration

Už ste videli, ako sa aplikuje metóda Policy Iteration pre riešenie problému gridworld. Ďalej sa pozrieme na to, ako nájde riešenie metóda Value Iteration pre rovnaký problém. Aby porovnávanie bolo naozaj férové, použijeme úplne rovnaký svet.

## Value Iteration

Value Iteration (alebo *iterácia hodnôt*) môžeme vnímať ako zjednodušenú verziu metódy iterácie politiky s tým, že kým sme pri Policy Iteration mali niekoľko krokov, ktoré sa opakovali striedavo, pri Value Iteration existuje len evaluácia politiky, ktorá sa vykonáva opakovane až dovtedy, kým sa nenájde stabilný odhad. Následne sa vygeneruje finálna politika, ktorá - pri splnení predpokladov pre konvergenciu metódy - bude skoro optimálna.

Úplný pseudokód algoritmu nájdete na obrázku nižšie:

<img src="lab04/value_iteration.jpg" width="600">
<p style="text-align: center;">Zdroj: Sutton-Barto: Reinforcement Learning, 2nd ed., 2018</p>

Prediskutujte jednotlivé kroky a možnosti programovej implementácie algoritmu. Následne algoritmus aplikujeme pri riešení príkladu gridworld z minulého týždňa:

<img src="lab04/gridworld_mdp.jpg" width="600">

Príklad predstavuje svet *3x3* s cieľovou pozíciou v pravom hornom rohu, a s jednou pascou v strede sveta. K dispozícii sú štyri akcie: posun na sever, východ, juh a západ. Ak sa hráč dostane do cieľa, obdrží odmenu 10, ak spadne do pasce, tak -10. V oboch prípadoch sa hra ukončí. Pre ostatné kroky dostane agent odmenu -1.

Dolná a pravá časť sveta je úplne deterministická, pričom na pozíciach uznačených svetlomodrou farbou fúka silný vietor, ktorý môže agenta posunúť na východ aj keď sa pohybuje iným smerom. Pravdepodobnosť pohybu vo vybranom smere je v týchto prípadoch $0.6$, pravdepodobnosť posunutia na východ je $0.4$. Ak agent vyberie pohyb na východ, určite sa tam dostane. Ak sa pohybuje na západ, ostane na svojej pôvodnej pozícii (vietor a jeho pohyb sa rušia).

Discount factor $\gamma = 0.8$ a $\theta = 1.0$. Prvý odhad hodnoty pre každý stav bude 0: $V_0 (s) = 0 \ \ \  \forall \ \ s \in \mathcal{S}$.

## Iterácia hodnôt

V prvej časti sa algoritmus snaží nájsť stabilný odhad hodnôt jednotlivých stavov, pričom platí:

$V(s) = \underset{a}{max} \sum_{s', r} p(s', r|s, a) \left [ r + \gamma \cdot V(s') \right ]$,

kde $p(s', r|s, a)$ je pravdepodobnosť toho, že po vykonaní akcie $a$ zo stavu $s$ sa prostredie presunie do stavu $s'$ a agent obdrží odmenu $r$.

Aktualizovaný odhad hodnoty stavu vieme určiť tak, že vypočítame hodnotu stavu pri vykonaní jednotlivých akcií, a následne si vyberieme tú najvyššiu hodnotu. Pre stav $s_{11}$ to bude:

$V^N (s_{11}) = p(s_{11}, r_{s11}|s_{11}, N) \left [ r_{s11} + \gamma \cdot V(s_{11}) \right ] + p(s_{12}, r_{s12}|s_{11}, N) \left [ r_{s12} + \gamma \cdot V(s_{12}) \right ] = 0.6 \cdot \left [ -1 + 0.8 \cdot 0 \right ] + 0.4 \cdot \left [ -1 + 0.8 \cdot 0 \right ] = -1$

$V^E (s_{11}) = p(s_{12}, r_{s12}|s_{11}, E) \left [ r_{s12} + \gamma \cdot V(s_{12}) \right ] = 1 \cdot \left [ -1 + 0.8 \cdot 0 \right ] = -1$

$V^S (s_{11}) = p(s_{21}, r_{s21}|s_{11}, S) \left [ r_{s21} + \gamma \cdot V(s_{21}) \right ] + p(s_{12}, r_{s12}|s_{11}, N) \left [ r_{s12} + \gamma \cdot V(s_{12}) \right ] = 1 \cdot \left [ -1 + 0.8 \cdot 0 \right ] + 0.4 \cdot \left [ -1 + 0.8 \cdot 0 \right ] = -1$

$V^W (s_{11}) = p(s_{11}, r_{s11}|s_{11}, W) \left [ r_{s11} + \gamma \cdot V(s_{11}) \right ] = 1 \cdot \left [ -1 + 0.8 \cdot 0 \right ] = -1$

Potom:

$V(s_{11}) = -1$

$\Delta = max(0, |0 - (-1)|) = 1; \ \ \ \ \Delta = \theta$

**Úloha:** Vypočítajte hodnoty pre zvyšné stavy (okrem $s_{13}$ a $s_{22}$) na základe nižšie uvedených hodnôt. Použite inline prístup.

$V(s_{11}) = 7.0$

$V(s_{12}) = 10.0$

$V(s_{13}) = 0.0$

$V(s_{21}) = -1.24$

$V(s_{22}) = 0.0$

$V(s_{23}) = 10.0$

$V(s_{31}) = -1.8$

$V(s_{32}) = 4.6$

$V(s_{33}) = 7.0$

## Generovanie výslednej politiky

Na rozdiel od metódy Policy Iteration, kde sa politika vygeneruje po každej iterácii, Value Iteration nám vygeneruje politiku až na konci behu, teda až potom, čo sa ustália odhadované hodnoty stavov. Spôsob výberu je skoro rovnaký ako výpočet hodnoty stavu, avšak výsledok nie je hodnota stavu pre dvojicu stav-akcia, ale samotná akcia:

$\pi (s) = \underset{a}{argmax} \sum_{s', r} p(s', r|s, a) \left [ r + \gamma \cdot V(s') \right ]$.

Ak odhad hodnôt stavov konverguje, potom vyprodukovaná politika bude (skoro) optimálna. V prípade nášho problému politiku by mali tvoriť akcie, ktoré hráča posunú viac smerom k cieľovej pozícii z jednotlivých stavov, napr.:

$\pi (s_{11}) = \{E\}$; $\pi (s_{12}) = \{E\}$; $\pi (s_{21}) = \{N\}$; $\pi (s_{23}) = \{N\}$; $\pi (s_{31}) = \{E\}$; $\pi (s_{32}) = \{E\}$; $\pi (s_{33}) = \{N\}$

## Implementácia

Overte správnosť vašich výpočtov pomocou [ukážkovej implementácie metódy Value Iteration](lab04/value_iteration.py) a spozorujte, ako sa metóda dopracuje k optimálnej politiky.

**Domáca úloha:** Upravte kód tak, aby implementoval sweep stratégiu.